In [1]:
from egoexo_dataloader_2 import EgoExoDataset
import os
import numpy as np
import torch
import random
from torch.utils.data import DataLoader

In [2]:
dataset_path = '/media/thibault/T5 EVO/Datasets/Ego4D/'
train_dataset = EgoExoDataset(dataset_path, os.path.join(dataset_path, 'takes.json'), split = "train", skill=True, get_frames=False, get_pose=True, get_hands_pose=False, frame_rate=3, transform=None)
val_dataset = EgoExoDataset(dataset_path, os.path.join(dataset_path, 'takes.json'), split = "val", skill=True, get_frames=False, get_pose=True, get_hands_pose=False, frame_rate=3, transform=None)

Pose data for a5e81f20-ea35-4f63-a908-11d89fc1294c not found.
Pose data for 3db860b7-a23a-4d9b-a988-4b656a608122 not found.
Pose data for 98f58f0f-53d6-4e41-bf41-d8d74ccbc37c not found.
Pose data for 38bb5f61-707d-4cf3-b1d1-a8260b7dfadc not found.
Pose data for 1ec1409c-a3fd-46ab-8698-99f9317e9f13 not found.
Pose data for 35823fca-da54-49a2-8d4c-365046031e2c not found.
Pose data for 76a7ea80-0a81-4b4f-983e-a7e36fa26685 not found.
Pose data for 7c0a813b-dce3-4ba0-92cc-a8e0a6f87581 not found.
Pose data for 6a90f588-8a69-4d1c-b071-85527fc98042 not found.
Pose data for 43d60820-1daa-472f-be4d-6802746df7d8 not found.
Pose data for e6f04a5d-0da7-42c9-8b27-4e240319047e not found.
Pose data for 699dde3f-cd23-4674-939e-42438faeb0da not found.
Pose data for b68f9254-913a-4aee-9ae8-29d1121dd892 not found.
Pose data for e1df2957-a10c-4bc0-ac91-4014551287f0 not found.
Pose data for be1eb1c1-a45f-4e8b-9ae2-3d8db0af7e59 not found.
Pose data for cc5a6be4-d030-4057-9ed5-1f895c6c9274 not found.
Pose dat

In [3]:
data = train_dataset.__getitem__(50)
data

{'ego': None,
 'exo': None,
 'pose': {'0': array([{'keypoints': array([[[ 324.78924298,  109.18517376],
                 [ 239.48237092,    0.        ],
                 [ 239.14007637,    0.        ],
                 [ 189.68002439,  305.32573508],
                 [ 674.81190048,    0.        ],
                 [   0.        ,    2.65967957],
                 [   0.        , 1596.07089956],
                 [   0.        ,    0.        ],
                 [1895.65166719,    0.        ],
                 [ 724.19771439,  524.21631527],
                 [ 772.65556683,    0.        ],
                 [2273.14422583,  896.69795007],
                 [   0.        , 1194.04559673],
                 [ 638.85083048,  354.18346093],
                 [1246.29705644,  109.61015297],
                 [1567.68725515,  291.61553776],
                 [1240.30220035,  161.4711899 ]]]), 'keypoint_scores': array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
                0.

In [ ]:
print(len(train_dataset), len(val_dataset))

562 148


In [11]:
skeletons = list(data["pose"].values())


for skeleton in skeletons:
    for frame in skeleton:
        print(np.min(frame["keypoints"]))
        

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
806.9889326974848
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
977.6647086392643
976.1833602899011
980.8849871969322
0.0
998.601866491738
999.2158072104893
998.6561436384526
1000.7538677670885
1002.7038410535731
1015.3759564976514
1018.2303965667264
1020.0368429834709
1019.0006538816368
1017.6920906708452
1018.2766597913944
1024.0568736550058
1027.3004150391
1026.571852340533
1021.9946134016747
1025.9010558108273
1029.8149573991739
1034.3537228769453
0.0
1032.8347540219404
0.0
1035.7521972656
1033.3742675781
1044.0394287109
0.0
1046.8957519531
0.0
1049.1307373047
1054.6950683594
1056.0291748047
1055.2081298828
1053.69

In [12]:
from mmaction.apis import inference_skeleton, init_recognizer
from mmengine import Config
import torch
torch.cuda.empty_cache()
config_path = "/home/thibault/Documents/Code/pckg/mmaction2/configs/skeleton/posec3d/slowonly_r50_8xb16-u48-240e_ntu60-xsub-keypoint.py"
checkpoint_path = "/home/thibault/Documents/Code/pckg/mmaction2/checkpoints/slowonly_r50_8xb16-u48-240e_ntu60-xsub-keypoint_20220815-38db104b.pth" # can be a local path   # you can specify your own picture path

cfg = Config.fromfile(config_path)



# build the model from a config file and a checkpoint file
skeleton_model = init_recognizer(cfg, checkpoint_path, device="cuda:0")


Loads checkpoint by local backend from path: /home/thibault/Documents/Code/pckg/mmaction2/checkpoints/slowonly_r50_8xb16-u48-240e_ntu60-xsub-keypoint_20220815-38db104b.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: cls_head.fc_cls.weight, cls_head.fc_cls.bias



In [14]:
import torch

def collate_pose_and_label_with_padding(batch):
    """
    batch: list of samples, each a dict with:
      - 'pose':  np.ndarray (dtype=object) length N_i of view‐dicts
      - 'label': int

    Returns:
      padded_keypoints: FloatTensor (B, V_max, K, 2)
      padded_scores:    FloatTensor (B, V_max, K)
      view_mask:        BoolTensor  (B, V_max)
      labels:           LongTensor  (B,)
    """
    B = len(batch)
    # 1) labels
    labels = torch.tensor([s['label'] for s in batch], dtype=torch.long)

    # 2) collect pose-arrays and find max views
    pose_arrays = [s['pose'] for s in batch]            # list of np.object arrays
    num_views   = [len(pa) for pa in pose_arrays]       # N_i
    V_max       = max(num_views)

    # 3) get joint-count K (and dim=2) from the first view of first sample
    sample0_view0 = pose_arrays[0]['0'][0]
    _, K, dim = sample0_view0['keypoints'].shape        # (1, K, 2)

    # 4) allocate padded tensors
    padded_keypoints = torch.zeros(B, V_max, K, dim, dtype=torch.float)
    padded_scores    = torch.zeros(B, V_max, K,    dtype=torch.float)
    view_mask        = torch.zeros(B, V_max,        dtype=torch.bool)

    # 5) fill in real views
    for b, pa in enumerate(pose_arrays):
        for v, view_dict in enumerate(pa.values()):
            # squeeze out that leading 1 if present
            kp = view_dict['keypoints']
            if kp.ndim == 3 and kp.shape[0] == 1:
                kp = kp[0]            # now (K, 2)
            scores = view_dict['keypoint_scores']  # (K,)

            padded_keypoints[b, v] = torch.from_numpy(kp)
            padded_scores[b, v]    = torch.from_numpy(scores)
            view_mask[b, v]        = True

    return padded_keypoints, padded_scores, view_mask, labels


In [15]:
import torch

def collate_pose_and_label_with_padding_2(batch):
    """
    batch: list of samples, each a dict with:kp
      - 'pose':  np.ndarray (dtype=object) length N_i of view‐dicts
      - 'label': int, V_max)
      labels:           LongTensor  (B,)
    """
    B = len(batch)
    # 1) labels
    labels = torch.tensor([s['skill'] for s in batch], dtype=torch.long)

    # 2) collect pose-arrays and find max views
    pose_arrays = [s['pose'] for s in batch]            # list of np.object arrays
    num_views   = [len(pa) for pa in pose_arrays]       # N_i
    V_max       = max(num_views)

    for sample in batch:
      if len(sample["pose"].keys()) < V_max:
        for i in range(len(sample["pose"].keys()), V_max):
          T = len(sample["pose"]["0"])
          sample["pose"][str(i)] = np.array(np.repeat({"keypoints": np.zeros((V_max, 17, 2)), "keypoint_scores": np.zeros((V_max, 17))}, T))
       
        
      
    
    return pose_arrays, labels


In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class MultiViewSkeletonClassifier(nn.Module):
    """
    PyTorch model for multi-view skeleton fusion and classification.

    Inputs:
        - Four skeleton feature tensors, each of shape (batch_size, seq_len=20, feat_dim=512).
    Output:
        - Logits for classification into num_classes.
    """
    def __init__(self, skeleton_model, feat_dim=512, seq_len=20, num_views=4, hidden_dim=512, num_classes=10):
        super(MultiViewSkeletonClassifier, self).__init__()
        self.num_views = num_views
        self.seq_len = seq_len
        self.feat_dim = feat_dim
        self.skeleton_model = skeleton_model
        
        # Define the attention head for per-view summary
        self.attn_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim // 2, 1)
        )
        
        

        # Learnable fusion weights for each view (softmax-normalized)
        self.view_weights = nn.Parameter(torch.ones(num_views))

        # Project concatenated features to hidden dimension
        self.fusion_proj = nn.Linear(feat_dim, hidden_dim)

        # Temporal convolutional layers
        self.conv1 = nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.conv2 = nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(hidden_dim)

        # Classification head
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim * seq_len, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, poses):
        """
        Args:
            views: list or tuple of length num_views, each tensor of shape (B, T, D)
        Returns:
            logits: tensor of shape (B, num_classes)
        """
        # Stack views: (V, B, T, D) -> (B, V, T, D)
        batch = []
        
        for sample in poses:
            views = []
            for view in sample.values():
                representation = inference_skeleton(self.skeleton_model, view,(1920,1080), test_pipeline=None)  # (B, T, D)
                views.append(representation)  # (B, T, D)
            batch.append(torch.stack(views))  # (B, T, D)
            
        x = torch.stack(batch).to(device='cuda:0')  # (B, V, T, D)
        B, V, T, D = x.shape
        
        # per‐view summary → score → masked softmax → fusion
        summary = x.mean(dim=2)              # (B, V, hidden)
        raw_w   = self.attn_head(summary).squeeze(-1) # (B, V)
        w       = F.softmax(raw_w)    # (B, V)
        fused   = (w.view(B,V,1,1) * x).sum(dim=1)


        # Project features
        fused = self.fusion_proj(fused)  # (B, T, hidden_dim)
        fused = F.relu(fused)

        # Prepare for temporal conv: (B, hidden_dim, T)
        fused = fused.permute(0, 2, 1)

        # Temporal convolutional block
        out = F.relu(self.bn1(self.conv1(fused)))
        out = F.relu(self.bn2(self.conv2(out)))

        # Flatten temporal features
        out = out.view(B, -1)

        # Classification head
        logits = self.fc(out)
        return logits


if __name__ == "__main__":
    # Example usage
    B, T, D = 8, 20, 512
    num_views = 4
    num_classes = 5
    multiview_model = MultiViewSkeletonClassifier(skeleton_model=skeleton_model, feat_dim=512, seq_len=T, num_views=num_views, hidden_dim=D, num_classes=num_classes).to('cuda:0')
    # Create dummy inputs
    views = [torch.randn(B, T, D) for _ in range(num_views)]
    results = multiview_model(([data["pose"], data["pose"]]))  # Forward pass
    print("Input shapes:", [view.device for view in views])  # expected (8, 20, 512)
    print(results)
     # expected (8, num_classes)


/tmp/ipykernel_131829/2794633667.py:74: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  w       = F.softmax(raw_w)    # (B, V)


Input shapes: [device(type='cpu'), device(type='cpu'), device(type='cpu'), device(type='cpu')]
tensor([[-3.9375e-02, -4.7466e-01,  8.9326e-02,  2.5757e-01, -1.0640e-01],
        [-3.6232e-03, -4.4340e-04, -9.7668e-02,  5.2915e-01, -3.3700e-02]],
       device='cuda:0', grad_fn=<AddmmBackward0>)


In [18]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.optim as optim

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Hyperparameters
num_epochs    = 30
learning_rate = 0.001
batch_size    = 8
num_classes   = 5

# Model
multiview_model = MultiViewSkeletonClassifier(
    skeleton_model=skeleton_model,
    feat_dim=512,
    seq_len=20,
    num_views=4,
    hidden_dim=512,
    num_classes=num_classes
).to(device)
model = multiview_model

# Loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)


# Dataloaders
train_loader = DataLoader(
    train_dataset,
    collate_fn=collate_pose_and_label_with_padding_2,
    batch_size=batch_size,
    shuffle=True
)
val_loader = DataLoader(
    val_dataset,  # assuming you have a separate validation dataset
    collate_fn=collate_pose_and_label_with_padding_2,
    batch_size=batch_size,
    shuffle=False
)

# Training + Validation loop
for epoch in range(num_epochs):
    # ---- TRAIN ----
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for views, labels in tqdm(train_loader, desc=f"Train Epoch {epoch+1}/{num_epochs}"):
        # if views is a list of tensors; otherwise .to(device)
        labels = labels.to(device)

        # forward + backward
        outputs = model(views)
        loss    = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # stats
        running_loss += loss.item()
        _, preds = outputs.max(1)
        total   += labels.size(0)
        correct += preds.eq(labels).sum().item()
        
    
    train_loss = running_loss / len(train_loader)
    train_acc  = 100. * correct / total

    # ---- VALIDATION ----
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total   = 0

    with torch.no_grad():
        for views, labels in tqdm(val_loader, desc=f"Val   Epoch {epoch+1}/{num_epochs}"):
            
            labels = labels.to(device)

            outputs = model(views)
            loss    = criterion(outputs, labels)

            val_loss += loss.item()
            _, preds = outputs.max(1)
            val_total   += labels.size(0)
            val_correct += preds.eq(labels).sum().item()

    val_loss_epoch = val_loss / len(val_loader)
    val_acc        = 100. * val_correct / val_total
    scheduler.step(val_loss_epoch)
    # ---- EPOCH SUMMARY ----
    print(
        f"Epoch {epoch+1}/{num_epochs} | "
        f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
        f"Val   Loss: {val_loss_epoch:.4f}, Val   Acc: {val_acc:.2f}%"
    )
    torch.save(model.state_dict(), f"multiview_model_epoch_{epoch+1}.pth")
print("Training complete.")



Train Epoch 1/30:   0%|          | 0/71 [00:00<?, ?it/s]/tmp/ipykernel_131829/2794633667.py:74: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  w       = F.softmax(raw_w)    # (B, V)
Val   Epoch 1/30: 100%|██████████| 19/19 [05:21<00:00, 16.91s/it]


Epoch 1/30 | Train Loss: 0.8726, Train Acc: 86.65% | Val   Loss: 0.5747, Val   Acc: 90.54%


Val   Epoch 2/30: 100%|██████████| 19/19 [04:57<00:00, 15.65s/it]


Epoch 2/30 | Train Loss: 0.3205, Train Acc: 92.17% | Val   Loss: 0.3369, Val   Acc: 90.54%


Val   Epoch 3/30: 100%|██████████| 19/19 [05:18<00:00, 16.77s/it]


...

Epoch 28/30 | Train Loss: 0.0145, Train Acc: 99.47% | Val   Loss: 6.9321, Val   Acc: 68.24%


Val   Epoch 29/30: 100%|██████████| 19/19 [05:12<00:00, 16.46s/it]


Epoch 29/30 | Train Loss: 0.0117, Train Acc: 99.64% | Val   Loss: 8.9647, Val   Acc: 50.68%


Val   Epoch 30/30: 100%|██████████| 19/19 [05:17<00:00, 16.71s/it]

Epoch 30/30 | Train Loss: 0.0108, Train Acc: 99.64% | Val   Loss: 8.3950, Val   Acc: 41.22%
Training complete.
